# CS202: Compiler Construction

## In-class Exercises, Week of 01/23/2023

----

# Part 1: Lvar

## Question 1

Write an interpreter `eval_lvar` for the `Lvar` language. Reference the grammar: Figure 2.2 (page 14) in the textbook, and our simplified AST in the [Assignment 2 Description](https://github.com/jnear/cs202-compiler-construction/blob/master/homework/hw2.md).

In [1]:
from typing import Dict
from cs202_support.python import *

# op ::= "add"
# Stmt ::= Assign(x, Expr) | Print(Expr)
# Expr ::= Var(x) | Constant(n) | Prim(op, [Expr])
# LVar ::= Program([Stmt])

"""
Interpreter: function that takes a program and evaluates it
Compiler: function that takes a program and returns a new program
"""

def eval_lvar(program: Program):
    def eval_stmt(s: Stmt, env: Dict[str, int]):
        match s:
            case Assign(x, e):
                env[x] = eval_expr(e, env)
            case Print(e):
                print(eval_expr(e, env)) 

    def eval_expr(e: Expr, env: Dict[str, int]) -> int:
        match e:
            case Var(x):
                return env[x]
            case Constant(n):
                return n
            case Prim("add", [e1,e2]):
                val1 = eval_expr(e1, env)
                val2 = eval_expr(e2, env)
                return val1 + val2

    env = {}
    for s in program.stmts:
        eval_stmt(s, env)

In [2]:
# Test case for interpreter
program = """
x = 5
y = 6
print(x + y)"""

eval_lvar(parse(program))

11


----
# PART 2: Remove Complex Operands

## Question 2

Consider this translation of an expression to assembly language. What is wrong with it?

In [3]:
python = """
x = 1 + 2 + 3
"""

# In assembly language it is not possible to nest expressions 
asm = """
movq $2, %rax
addq $1, (addq $3, %rax)
"""

Assembly language doesnt support nested arguments, arguments to an instruction need to be atomic
- Register
- Memory location
- Constant

## Question 3

Which AST nodes in the language `Lvar` are **atomic**?

Atomic expressions are:
- Var
- Constant

Prim is not atomic because it may have some expressions inside of it

## Question 4

Why do we need this pass? What is the form of its output?

This pass un-nests expressions so that the assembly language can handle them. The output is a program that has no nested expressions.

The output of this pass will have primitives that are atomic expressions

This formal is called A-normal form (or monadic normal form)

## Question 5

Convert the program from earlier into A-normal form.

In [4]:
python = """
x = 1 + 2 + 3
"""

In [5]:
python_anf = """
temp1 = 1 + 2
x = temp1 + 3
"""

## Question 6

Describe a recursive procedure to perform the *remove-complex-opera* pass. Reference section 2.4 in the textbook.

# Structure will follow the structure of the grammar

- `rco_exp(exp) -> exp`
  - For var: `rco_var(var) -> var`
  - For const: `rco_const(const) -> const`
  - For prim: 
    - Create a temp variable and assign it to the result of the primitive
    - Store the temp variable in the environment
- rco_stmt(stmt) -> stmt
- rco_stmts(stmts) -> stmts

----
# Part 3: Select-instructions

The select-instructions pass transforms a sequence of statements into X86 assembly instructions.

## Question 7

**(Not included in Jan 27 in class exercise)**

Convert the following `Lvar` code into a psuedo-x86 assembly program.

```
Module([Assign([Name("y")], Constant(5)),
        Assign([Name("x")], Name("y")),
        Expr(Call(Name("print"), [Name("x")]))
])
```

## Question 8

**(Not included in Jan 27 in class exercise)**

Describe the structure of select-instructions.